## Connect to ICW:

In [1]:
import cadspy

In [2]:
username = str(input('User number'))
icw = cadspy.DatabaseConnection(system='ICW', user=username)

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:salmon;font-family:Verdana,sans-serif;font-size:16px;">

<font size="4">Can't connect to ICW? Instructions on how to get access to it in the links below:

</font>
</div>

- Step 1: [Requesting Access to GitHub](https://baplc.sharepoint.com/sites/ask/SitePages/Requesting-Access-to-GitHub.aspx)

Once your access to GitHub has been aproved, you need to:

- Step 2: [Request access to British-Ent GitHub organisation](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/Corporate_Directory_git.md)

And

- Step 3: [Setup Sagemaker Studio to access GitHub](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/sagemaker_to_github_setup.md)

<br>

Below are some packages to get you started. You don't have to use them but you may find them useful!

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

In [4]:
# diplay all rows and cols when using 'dataframe'.head() or 'dataframe'.tail()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<br>

### Data

#### S19 Lounge Eligibility data

In [5]:
query = """

sel * from LDB_SBOX_OR.HACKATHON_OPS_LOUNGE_ELIGIBILITY

"""

df_lounge_eligibility = icw.queryToDataframe(query)

In [6]:
df_lounge_eligibility.head(2)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13


In [7]:
df_lounge_eligibility.shape

(1534750, 12)

In [8]:
# a look to a particular flight number and date

#mask = (df_lounge_eligibility['DISCHARGE_STN_CD'] == 'GCM   ') & (df_lounge_eligibility['GMT_UPLIFT_DT'] == dt.date(2019,9,12) )

#df_lounge_eligibility[mask]

#### S19 Flight info

In [9]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_FLIGHT_INFO

"""

df_flight_info = icw.queryToDataframe(query)

In [10]:
df_flight_info.head(2)

,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE
0,2023-04-30 20:25:00,2023-04-30 20:34:00,BA,57,LHR,5,JNB,JNB,388,A8,LHRJNB
1,2019-05-18 12:15:00,2019-05-18 12:16:00,BA,632,LHR,5,ATH,ATH,320,R3,LHRATH


#### Station Code Decode

In [11]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_COUNTRY_DECODE

"""

df_country = icw.queryToDataframe(query)

In [12]:
df_country.head(2)

,ROUTE,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM
0,LHRINV,GB,United Kingdom and Northern Ireland,UK,UK
1,LHRSVO,RU,Russia in Europe,EASTERN EUROPE,EUROPE EXC UK


#### Aircraft Type

In [13]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_AC_TYPE

"""

df_acft_typ = icw.queryToDataframe(query)

In [14]:
df_acft_typ.head(2)

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,0,24,0,132
1,332,W9,WB,0,30,21,235


In [15]:
df_acft_typ.shape

(35, 7)

In [107]:
df_acft_typ.head(5)

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,0,24,0,132
1,332,W9,WB,0,30,21,235
2,339,W9,WB,0,30,21,234
3,777,K7,WB,0,48,24,203
4,781,X8,WB,7,49,35,165


Merging df_country with route haul and business/leisure data from reference tables in ICW.

They will be merged into df_lounge_eligibility for further analysis.

In [17]:
# A corporate_reference table just for flight numbers
# and their descriptions.
# To be matched on flight number and date (inner join probably)
# Checked with TLV and AMM, both correct (LH, SH)
query = """

select FM_FLT_NO, TO_FLT_NO, EFF_DT, EXP_DT, LONGHAUL_IND, SHORTHAUL_IND,
DOMESTIC_IND, RANGE_DESC_TXT

from CRT_FLIGHT_NO_RANGES

WHERE CODESHARE_IND = 'N' AND FRANCHISE_IND = 'N' 

"""

df_flt_no_ranges = icw.queryToDataframe(query)

In [18]:
df_flt_no_ranges.head(10)

,FM_FLT_NO,TO_FLT_NO,EFF_DT,EXP_DT,LONGHAUL_IND,SHORTHAUL_IND,DOMESTIC_IND,RANGE_DESC_TXT
0,8000,8019,2019-03-31,2021-08-12,Y,N,N,Openskies for LEVEL France
1,2530,2899,2022-10-30,2022-10-31,N,Y,N,LGW Euroflyer Prime Shorthaul
2,2530,2825,2023-10-18,2023-10-18,N,Y,N,LGW Euroflyer Prime LGW Shorthaul
3,2360,2369,2022-10-30,2022-10-31,N,Y,N,BA Cityflyer Prime
4,3000,3199,2005-01-01,2021-10-30,N,N,N,BA Cargo Interline Services (Robbie Newport x8...
5,2826,2827,2023-10-18,2023-10-18,N,Y,N,LGW Euroflyer Prime LGW Shorthaul
6,2210,2217,2019-03-31,2021-10-30,N,Y,N,BA Cityflyer (CJ) (London City services Shorth...
7,3300,3319,2005-01-01,2021-10-30,Y,N,N,BA wet lease and shared longhaul freighter ser...
8,9278,9279,2022-03-27,2022-10-29,N,N,N,Euroflyer Engineering
9,2218,2229,2019-03-31,2021-10-30,N,Y,N,BA Cityflyer (CJ) (London City Services)


In [19]:
# For more granular details of route type, we can also use RevMan definitions.

query = """

SELECT DEP_STN_CD, ARR_STN_CD, GCD_KM, REVMAN_HUB_TXT, 
REVMAN_AREA_TXT, ROUTE_CAT_TXT
FROM REF_ROUTE

WHERE DEP_STN_CD = 'LHR' AND REVMAN_HUB_TXT <> 'Unknown'

"""

df_route_categories = icw.queryToDataframe(query)

<br>

### Pre-processing

*Hint:* It is always worth checking the format of each of the columns in your dataframes before trying to do any work with them. To do so, you can make use of the `headers_and_first_row` function below.


In [20]:
def headers_and_first_row(df):
    '''
    print headers and first row of a df to deal with data types
    '''
    
    headers = df.columns
    first_row = []

    for col in headers:
        first_row.append(df[col][0])
    
    dictionary = dict( zip( headers, first_row) )

    return dictionary

In [21]:
# applying headers_and_first_row to df_lounge_eligibility
format_df = headers_and_first_row(df_lounge_eligibility)
format_2 = headers_and_first_row(df_flight_info)

In [22]:
# Note that some columns have blank spaces!
format_df
format_2

{'GMT_PLND_DEP_TS': Timestamp('2023-04-30 20:25:00'),
 'GMT_ACT_DEP_TS': Timestamp('2023-04-30 20:34:00'),
 'OPG_ALN_CD': 'BA    ',
 'OPG_FLT_NO': 57,
 'ACT_DEP_STN_CD': 'LHR   ',
 'ACT_DEP_TML_CD': '5 ',
 'PLND_ARR_STN_CD': 'JNB   ',
 'ACT_ARR_STN_CD': 'JNB   ',
 'IATA_AC_TYP_CD': '388   ',
 'ACT_AC_TYP_CD': 'A8  ',
 'ROUTE': 'LHRJNB      '}

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 1:** Pre-process **all** the tables above (df_lounge_eligibility, df_flight_info, df_country, df_acft_typ).

</font>

</ol>
</div>

We will first strip all frames of spaces. We will then go through each frame to properly prepare it.

We will then load some additional datasets that will be useful for analysis.

In [23]:
# Stripping strings

tables = [df_lounge_eligibility,df_flight_info,df_country,df_acft_typ,df_flt_no_ranges,df_route_categories]

# Define a function that fine all string fields and remove all blak spaces
def data_cleaning_string(df):
    # Get names of all fields in a dataframe
    fields = df.columns
    # loop for all fields, if data type is string then remove blank spaces
    for f in fields:
        if type(df[f][0]) == str:
            df[f] = df[f].str.strip()
    return df

for i,t in enumerate(tables):
    tables[i] = data_cleaning_string(t)

In [24]:
df_flight_info[df_flight_info['ACT_ARR_STN_CD'] == 'BGI'].head(10)

,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE
241,2023-04-28 10:30:00,2023-04-28 10:49:00,BA,255,LHR,5,BGI,BGI,777,M7,LHRBGI
3522,2023-08-25 10:30:00,2023-08-25 10:30:00,BA,255,LHR,5,BGI,BGI,777,M7,LHRBGI
3889,2023-08-24 10:30:00,2023-08-24 10:27:00,BA,255,LHR,5,BGI,BGI,777,M7,LHRBGI
5322,2023-09-24 10:30:00,2023-09-24 10:29:00,BA,255,LHR,5,BGI,BGI,777,M7,LHRBGI
5541,2023-09-23 10:30:00,2023-09-23 10:41:00,BA,255,LHR,5,BGI,BGI,777,M7,LHRBGI
6161,2023-10-22 10:30:00,2023-10-22 11:25:00,BA,255,LHR,5,BGI,BGI,777,M7,LHRBGI
7026,2023-10-12 10:30:00,2023-10-12 10:35:00,BA,255,LHR,5,BGI,BGI,777,M7,LHRBGI
7031,2023-07-15 10:30:00,2023-07-15 10:26:00,BA,255,LHR,5,BGI,BGI,777,M7,LHRBGI
8064,2023-09-02 10:30:00,2023-09-02 10:55:00,BA,255,LHR,5,BGI,BGI,777,M7,LHRBGI
9237,2023-04-07 10:30:00,2023-04-07 10:28:00,BA,255,LHR,5,BGI,BGI,77W,G7,LHRBGI


In [25]:
# count duplicates of flight_info
test_flight_dups = df_flight_info[df_flight_info.duplicated(subset=['GMT_PLND_DEP_TS','OPG_FLT_NO'],keep=False)]
test_flight_dups.sort_values(['PLND_ARR_STN_CD','GMT_ACT_DEP_TS','OPG_FLT_NO']).head(5)

,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE
11564,2023-07-09 06:50:00,2023-07-09 07:50:00,BA,310,LHR,3,ALG,LHR,320,T3,LHRALG
12140,2023-07-09 06:50:00,2023-07-09 15:11:00,BA,310,LHR,3,ALG,ALG,320,T3,LHRALG
39912,2019-08-14 07:55:00,2019-08-14 07:55:00,BA,774,LHR,5,ARN,LHR,32A,H3,LHRARN
40488,2019-08-14 07:55:00,2019-08-14 09:58:00,BA,774,LHR,5,ARN,ARN,32A,H3,LHRARN
27935,2023-05-06 10:30:00,2023-05-06 12:57:00,BA,255,LHR,5,BGI,LHR,777,M7,LHRBGI


In [26]:
# Removing duplucates of flight_info

# Sort the table
df_flight_info = df_flight_info.sort_values(['GMT_PLND_DEP_TS','OPG_FLT_NO','GMT_ACT_DEP_TS'])
# Remove all duplicates and keep the first rows of all duplicated
df_flight_info = df_flight_info.drop_duplicates(subset = ['OPG_FLT_NO','GMT_PLND_DEP_TS'],keep = "first")


In [27]:
df_flight_info.shape

(136180, 11)

In [28]:
# 2. Flight info
# Ian's code to clean dataset - identify latest departure date for
# duplicated flights and eliminate it.
df_flight_info['GMT_PLND_DEP_DT'] = df_flight_info['GMT_PLND_DEP_TS'].dt.date
df_flight_info['GMT_PLND_DEP_TIME'] = df_flight_info['GMT_PLND_DEP_TS'].dt.time

In [29]:
# 3. Country
df_country['DEP_STN_CD'] = df_country['ROUTE'].str.slice(0,3)
df_country['ARR_STN_CD'] = df_country['ROUTE'].str.slice(3,6)


In [30]:
df_flight_info.head(1)

,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME
10910,2019-03-31 05:15:00,2019-03-31 05:08:00,BA,472,LHR,3,BCN,BCN,320,A3,LHRBCN,2019-03-31,05:15:00


In [106]:
# 4. Aircraft type
df_acft_typ.sort_values('FIRST_SEATS_QTY').tail(10)
not_in_af_typ = df_flight_info[~df_flight_info['ACT_AC_TYP_CD'].isin(df_acft_typ['ACT_AC_TYP_CD'])]
ac_missing_counts = not_in_af_typ['ACT_AC_TYP_CD'].value_counts()
print(ac_missing_counts)
df_acft_typ.head(5)

T7    28
U7     1
K3     1
Name: ACT_AC_TYP_CD, dtype: int64


,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,0,24,0,132
1,332,W9,WB,0,30,21,235
2,339,W9,WB,0,30,21,234
3,777,K7,WB,0,48,24,203
4,781,X8,WB,7,49,35,165


In [32]:
df_lounge_eligibility.shape

(1534750, 12)

In [33]:
# Joining flight_no_ranges with lounge eligibility
# df_merged_a = pd.merge(df_lounge_eligibility,df_flt_no_ranges,how='left',
#                       left_on=['OPERATING_FLT_NO'],right_on=['FM_FLT_NO'])
# print(df_merged_a.shape)
# df_merged_a = df_merged_a[(df_merged_a['GMT_UPLIFT_DT'] >= df_merged_a['EFF_DT'])&
#                     (df_merged_a['GMT_UPLIFT_DT']<=df_merged_a['EXP_DT'])]
# df_merged_a.drop(['FM_FLT_NO','TO_FLT_NO','EFF_DT','EXP_DT'],axis=1,inplace=True)
# print(df_merged_a.shape)

In [34]:
# df_merged_a.sort_values(['Skew_Id']).head(2)
# # success

# df_merged_a.shape

In [35]:
# df_merged_b = pd.merge(df_merged_a,df_route_categories,
#                       left_on=['UPLIFT_STN_CD','DISCHARGE_STN_CD'],
#                       right_on=['DEP_STN_CD','ARR_STN_CD'],
#                       how='left')

In [36]:
# # check for success
# df_merged_b.head(2)

# df_merged_b.shape

In [37]:
# df_lounge_eligibility = df_merged_b.copy()

# df_lounge_eligibility.head(5)

# df_lounge_eligibility.shape

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 2:** Join the tables below
   
    - df_flight_info
    - df_country
    - df_acft_typ
    
to the table df_lounge_eligibility to generate a final dataset.

</font>

</ol>
</div>

PLUS for us, join df_flight_no_ranges and df_route_categories.

In [38]:
# df_lounge_eligibility.head(5)

In [39]:
df_lounge_elig_flight_info = pd.merge(df_lounge_eligibility,# left table
                                     df_flight_info, # right table
                                     left_on = ['OPERATING_AIRLINE_CD','OPERATING_FLT_NO','GMT_UPLIFT_DT'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['OPG_ALN_CD','OPG_FLT_NO','GMT_PLND_DEP_DT'] , # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

df_lounge_elig_flight_info.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00


In [40]:
# your code here!
df_lounge_elig_flight_info.shape

(1534750, 25)

Get rows with any null values

In [41]:
df_lounge_elig_flight_info[df_lounge_elig_flight_info.isna().any(axis=1)].shape

(1183, 25)

In [42]:
df_lounge_elig_flight_info.loc[df_lounge_elig_flight_info['Skew_Id'] == '2023-10-1483']

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME


In [43]:
df_lounge_elig_flight_info[df_lounge_elig_flight_info.isna().any(axis=1)].sort_values(by='GMT_UPLIFT_DT',ascending=False).head(10)
# Why do some flights not match up. Manual investigation with FlightRadar 24 shows many of the most recent ones were cancelled flights.
# But very odd that only some of rows were retained - appears to be only 1 or 2 pax though. So probs best to remove.
# I think this is reflected in real data too on e.g. Ops Dashboard. Cancelled flights will often show as 1 or 2 people, presumably where flight was cancelled ages ago and one person didn't rebook or get moved.

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME
625082,2023-10-14832,BA,832,2023-10-14,LHR,DUB,M,M,,,Not eligible,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133379,2023-10-13446,BA,446,2023-10-13,LHR,AGP,M,M,,,Not eligible,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1483884,2023-10-13304,BA,304,2023-10-13,LHR,CDG,C,C,,,Tier 3,2,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1483264,2023-10-13304,BA,304,2023-10-13,LHR,CDG,C,C,Silver,SAPP,Tier 3,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208143,2023-10-13578,BA,578,2023-10-13,LHR,VCE,C,C,,,Tier 3,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208719,2023-10-13578,BA,578,2023-10-13,LHR,VCE,M,M,,SAPP,Tier 3,2,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209295,2023-10-13578,BA,578,2023-10-13,LHR,VCE,C,C,,EMER,Tier 2,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222151,2023-10-13828,BA,828,2023-10-13,LHR,DUB,M,M,,,Not eligible,6,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
760816,2023-10-131414,BA,1414,2023-10-13,LHR,BHD,M,M,,,Not eligible,2,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1099412,2023-10-13460,BA,460,2023-10-13,LHR,MAD,C,C,,EMER,Tier 2,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df_flight_info[df_flight_info['OPG_FLT_NO'] == 8642].head(10)

,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME


### Merge `df_country` to table

In [45]:
[headers_and_first_row(d) for d in [df_lounge_elig_flight_info,df_country]]

[{'Skew_Id': '2023-08-05618',
  'OPERATING_AIRLINE_CD': 'BA',
  'OPERATING_FLT_NO': 618,
  'GMT_UPLIFT_DT': datetime.date(2023, 8, 5),
  'UPLIFT_STN_CD': 'LHR',
  'DISCHARGE_STN_CD': 'OLB',
  'BOOKED_CABIN_CD': 'C',
  'TRAVEL_CABIN_CD': 'C',
  'BA_PAX_TIER': 'Gold For Life',
  'ONEWORLD_TIER': 'EMER',
  'Lounge_eligibility_tier': 'Tier 2',
  'pax': 2,
  'GMT_PLND_DEP_TS': Timestamp('2023-08-05 14:25:00'),
  'GMT_ACT_DEP_TS': Timestamp('2023-08-05 14:57:00'),
  'OPG_ALN_CD': 'BA',
  'OPG_FLT_NO': 618.0,
  'ACT_DEP_STN_CD': 'LHR',
  'ACT_DEP_TML_CD': '3',
  'PLND_ARR_STN_CD': 'OLB',
  'ACT_ARR_STN_CD': 'OLB',
  'IATA_AC_TYP_CD': '319',
  'ACT_AC_TYP_CD': 'M4',
  'ROUTE': 'LHROLB',
  'GMT_PLND_DEP_DT': datetime.date(2023, 8, 5),
  'GMT_PLND_DEP_TIME': datetime.time(14, 25)},
 {'ROUTE': 'LHRINV',
  'COUNTRY_CD': 'GB',
  'COUNTRY_NM': 'United Kingdom and Northern Ireland',
  'CORP_GEOG_CTRY_GRP_NM': 'UK',
  'CORP_GEOG_CONTINENT_NM': 'UK',
  'DEP_STN_CD': 'LHR',
  'ARR_STN_CD': 'INV'}]

In [46]:
df_lounge_elig_flight_info_country = pd.merge(df_lounge_elig_flight_info,# left table
                                     df_country, # right table
                                     left_on = ['PLND_ARR_STN_CD'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['ARR_STN_CD'] , # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

# df_lounge_elig_flight_info = df_lounge_elig_flight_info.drop_duplicates()

df_lounge_elig_flight_info_country.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00,LHRSTR,DE,Germany,WEST EUROPE,EUROPE EXC UK,LHR,STR
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00,LHROSL,NO,Norway,SCANDINAVIA,EUROPE EXC UK,LHR,OSL
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00,LHRNCE,FR,France,WEST EUROPE,EUROPE EXC UK,LHR,NCE
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00,LHRCAI,EG,Egypt,NORTH AFRICA,AFRICA,LHR,CAI


In [47]:
df_lounge_elig_flight_info_country.shape

(1534750, 32)

Check for null values

In [48]:
df_lounge_elig_flight_info_country[df_lounge_elig_flight_info_country['COUNTRY_CD'].isna()].head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD
28,2023-06-25516,BA,516,2023-06-25,LHR,OPO,C,C,Silver,SAPP,Tier 3,4,2023-06-25 06:40:00,2023-06-25 06:34:00,BA,516.0,LHR,5,OPO,OPO,32N,N3,LHROPO,2023-06-25,06:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,2023-10-06804,BA,804,2023-10-06,LHR,DLM,C,C,,SAPP,Tier 3,1,2023-10-06 10:20:00,2023-10-06 10:35:00,BA,804.0,LHR,5,DLM,DLM,32N,N3,LHRDLM,2023-10-06,10:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,2023-07-04310,BA,310,2023-07-04,LHR,ALG,C,C,,,Tier 3,9,2023-07-04 07:05:00,2023-07-04 08:06:00,BA,310.0,LHR,3,ALG,ALG,320,T3,LHRALG,2023-07-04,07:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,2023-06-03744,BA,744,2023-06-03,LHR,HER,M,M,Gold,EMER,Tier 2,1,2023-06-03 08:50:00,2023-06-03 09:32:00,BA,744.0,LHR,5,HER,HER,32N,N3,LHRHER,2023-06-03,08:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,2023-04-27424,BA,424,2023-04-27,LHR,NAP,M,C,Gold,EMER,Tier 2,1,2023-04-27 14:20:00,2023-04-27 15:00:00,BA,424.0,LHR,5,NAP,NAP,320,R3,LHRNAP,2023-04-27,14:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
# make a list of all the airports which are not in the country_df
# if doing with inner join, this should be empty.
unique_destinations_df_lounge_elig = df_lounge_elig_flight_info_country['DISCHARGE_STN_CD'].value_counts().reset_index()
unique_destinations_df_lounge_elig.columns = ['STN_CD','Count']

# get a list of all the station codes in the existing country dataframe and compare to the 
dests_df_country = df_country['ARR_STN_CD'].unique()
missing_dests = unique_destinations_df_lounge_elig[~unique_destinations_df_lounge_elig['STN_CD'].isin(dests_df_country)]
#print(missing_dests)
# This list shows all the missing destinations in the dataset. We could easily do this manually.
# But probs worth asking stakeholder.
# Note still 9 LHR arrivals with left joins all the way through.
missing_dests_list = list(missing_dests['STN_CD'])
print(missing_dests_list)

# copy to clipboard, only for ease of use in sql to get country codes missing in dataset.
#
column_for_copying = missing_dests['STN_CD'].apply(lambda x: f"'{x}'").astype(str).str.cat(sep=', ')
pd.Series(column_for_copying).to_clipboard(index=False,header=False)


['BER', 'JER', 'NAP', 'BDA', 'BGI', 'OPO', 'PDX', 'FLR', 'TIA', 'FNC', 'MLE', 'SKG', 'NUE', 'CVG', 'DLM', 'HER', 'DBV', 'ALG', 'SAW', 'PEG', 'BJV', 'RHO', 'PFO', 'LXS', 'ADB', 'PDL', 'SZG', 'SUF', 'GNB', 'CAG']


In [125]:
# obtain the missing decodes.
query = """
SELECT STN_CD, COUNTRY_CD, COUNTRY_NM, CORP_GEOG_CTRY_GRP_NM, CORP_GEOG_CONTINENT_NM
FROM REF_GEOG_LOC_HIERARCHY
WHERE STN_CD IN ('BER', 'JER', 'NAP', 'BDA', 'BGI', 'OPO', 'PDX',
 'FLR', 'TIA', 'FNC', 'MLE', 'SKG', 'NUE', 'CVG', 'DLM', 'HER', 'DBV', 'ALG', 
 'SAW', 'PEG', 'BJV', 'RHO', 'PFO', 'LXS', 'ADB', 'PDL', 'SZG', 'SUF', 'GNB', 'CAG')
"""

df_additional_country_decodes=icw.queryToDataframe(query)
df_additional_country_decodes.head(5)

,STN_CD,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM
0,CVG,US,United States,USA,NORTH AMERICA
1,ADB,TR,Turkey,MEDITERRANEAN,EUROPE EXC UK
2,JER,GB,United Kingdom and Northern Ireland,UK,UK
3,HER,GR,Greece,MEDITERRANEAN,EUROPE EXC UK
4,SZG,AT,Austria,WEST EUROPE,EUROPE EXC UK


In [ ]:
# join to complete dataset.


### `df_acft_typ`

In [50]:
df_lounge_elig_flight_info_country_acft_typ = pd.merge(
    df_lounge_elig_flight_info_country,
    df_acft_typ,
    left_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    right_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    how = 'left'
)

df_lounge_elig_flight_info_country_acft_typ.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NB,0.0,21.0,0.0,113.0
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00,LHRSTR,DE,Germany,WEST EUROPE,EUROPE EXC UK,LHR,STR,NB,0.0,21.0,0.0,113.0
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00,LHROSL,NO,Norway,SCANDINAVIA,EUROPE EXC UK,LHR,OSL,NB,0.0,20.0,0.0,150.0
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00,LHRNCE,FR,France,WEST EUROPE,EUROPE EXC UK,LHR,NCE,NB,0.0,16.0,0.0,119.0
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00,LHRCAI,EG,Egypt,NORTH AFRICA,AFRICA,LHR,CAI,WB,0.0,45.0,0.0,212.0


In [51]:
df_lounge_elig_flight_info_country_acft_typ.shape

(1534750, 37)

Check for null

In [52]:
df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ.isna().any(axis=1)].head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
28,2023-06-25516,BA,516,2023-06-25,LHR,OPO,C,C,Silver,SAPP,Tier 3,4,2023-06-25 06:40:00,2023-06-25 06:34:00,BA,516.0,LHR,5,OPO,OPO,32N,N3,LHROPO,2023-06-25,06:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NB,0.0,20.0,0.0,150.0
44,2023-10-06804,BA,804,2023-10-06,LHR,DLM,C,C,,SAPP,Tier 3,1,2023-10-06 10:20:00,2023-10-06 10:35:00,BA,804.0,LHR,5,DLM,DLM,32N,N3,LHRDLM,2023-10-06,10:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NB,0.0,20.0,0.0,150.0
67,2023-07-04310,BA,310,2023-07-04,LHR,ALG,C,C,,,Tier 3,9,2023-07-04 07:05:00,2023-07-04 08:06:00,BA,310.0,LHR,3,ALG,ALG,320,T3,LHRALG,2023-07-04,07:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NB,0.0,26.0,0.0,142.0
87,2023-06-03744,BA,744,2023-06-03,LHR,HER,M,M,Gold,EMER,Tier 2,1,2023-06-03 08:50:00,2023-06-03 09:32:00,BA,744.0,LHR,5,HER,HER,32N,N3,LHRHER,2023-06-03,08:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NB,0.0,20.0,0.0,150.0
89,2023-04-27424,BA,424,2023-04-27,LHR,NAP,M,C,Gold,EMER,Tier 2,1,2023-04-27 14:20:00,2023-04-27 15:00:00,BA,424.0,LHR,5,NAP,NAP,320,R3,LHRNAP,2023-04-27,14:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NB,0.0,28.0,0.0,139.0


In [53]:
df_acft_typ[df_acft_typ['ACT_AC_TYP_CD'] == 'M4']

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
20,319,M4,NB,0,21,0,113


In [54]:
# Same code as previous join to identify missing values and list them and their frequency.
# Can discuss with stakeholder as to their preferred strategy. Have retained for now, can easily drop columns with NaN later.
missing_final_left_joins = df_lounge_elig_flight_info_country_acft_typ['ACT_AC_TYP_CD'].value_counts().reset_index()
missing_final_left_joins.columns = ['Value','Count']

ac_types_original = df_acft_typ['ACT_AC_TYP_CD'].unique()
missing_ac = missing_final_left_joins[~missing_final_left_joins['Value'].isin(ac_types_original)]
print(missing_ac)


   Value  Count
30    T7    412
32    U7     11


### More cleaning for columns redundancy

In [55]:
headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)

{'Skew_Id': '2023-08-05618',
 'OPERATING_AIRLINE_CD': 'BA',
 'OPERATING_FLT_NO': 618,
 'GMT_UPLIFT_DT': datetime.date(2023, 8, 5),
 'UPLIFT_STN_CD': 'LHR',
 'DISCHARGE_STN_CD': 'OLB',
 'BOOKED_CABIN_CD': 'C',
 'TRAVEL_CABIN_CD': 'C',
 'BA_PAX_TIER': 'Gold For Life',
 'ONEWORLD_TIER': 'EMER',
 'Lounge_eligibility_tier': 'Tier 2',
 'pax': 2,
 'GMT_PLND_DEP_TS': Timestamp('2023-08-05 14:25:00'),
 'GMT_ACT_DEP_TS': Timestamp('2023-08-05 14:57:00'),
 'OPG_ALN_CD': 'BA',
 'OPG_FLT_NO': 618.0,
 'ACT_DEP_STN_CD': 'LHR',
 'ACT_DEP_TML_CD': '3',
 'PLND_ARR_STN_CD': 'OLB',
 'ACT_ARR_STN_CD': 'OLB',
 'IATA_AC_TYP_CD': '319',
 'ACT_AC_TYP_CD': 'M4',
 'ROUTE_x': 'LHROLB',
 'GMT_PLND_DEP_DT': datetime.date(2023, 8, 5),
 'GMT_PLND_DEP_TIME': datetime.time(14, 25),
 'ROUTE_y': 'LHROLB',
 'COUNTRY_CD': 'IT',
 'COUNTRY_NM': 'Italy',
 'CORP_GEOG_CTRY_GRP_NM': 'MEDITERRANEAN',
 'CORP_GEOG_CONTINENT_NM': 'EUROPE EXC UK',
 'DEP_STN_CD': 'LHR',
 'ARR_STN_CD': 'OLB',
 'WB_NB_CAT': 'NB',
 'FIRST_SEATS_QTY': 0.0

In [56]:
to_drop = ['GMT_PLND_DEP_TS','GMT_ACT_DEP_TS','OPG_ALN_CD','OPG_FLT_NO','ACT_DEP_STN_CD','PLND_ARR_STN_CD','ACT_ARR_STN_CD','ROUTE_y',
'DEP_STN_CD','ARR_STN_CD','GMT_PLND_DEP_DT']

df_lounge_elig_flight_info_country_acft_typ = df_lounge_elig_flight_info_country_acft_typ.drop(columns = to_drop,)

df_lounge_elig_flight_info_country_acft_typ.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,3,319,M4,LHROLB,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,3,319,M4,LHRSTR,16:35:00,DE,Germany,WEST EUROPE,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,5,32A,H3,LHROSL,18:25:00,NO,Norway,SCANDINAVIA,EUROPE EXC UK,NB,0.0,20.0,0.0,150.0
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,5,319,A4,LHRNCE,14:40:00,FR,France,WEST EUROPE,EUROPE EXC UK,NB,0.0,16.0,0.0,119.0
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,5,343,W9,LHRCAI,16:15:00,EG,Egypt,NORTH AFRICA,AFRICA,WB,0.0,45.0,0.0,212.0


In [57]:
df_lounge_elig_flight_info_country_acft_typ['ACT_DEP_TML_CD'].unique()

array(['3', '5', nan], dtype=object)

In [58]:
df_lounge_elig_flight_info_country_acft_typ.isnull().values.any()
# THERE ARE NA VALUES FOR AIRCRAFT TYPE.

True

In [59]:
# TEMPORARY, REMOVE BEFORE SUBMISSION!
#df_lounge_elig_flight_info_country_acft_typ.to_csv('df_final.csv')

<br><br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">Exercise 3: Based on S2019/S2023 data, provide a lookup table of Lounge eligibility assumptions that can be applied to a future schedule. To do so, answer each of the following questions in the Markdown cell provided below. 

- What level of granularity do you use?
- What metric do you use to come up with Lounge eligibility profiles?

    
Note 1: **Provide evidence for your assumptions.** This can be in the form of tables, graphs, correlation matrix, etc.
    
Note 2: Make use of the examples below to give structure to your answer. Feel free to attend the Hackathon Clinics if you have any questions. 
</font>


    
</ol>
</div>

**Reasoning (Example 1)**:


Assume:
- Data has been preprocessed.
- Data has been joined, and a final dataset has been created. This dataset is the result of joining the 4 tables.

The final dataset has been called `df_lounge_elig_flight_info_country_acft_typ`.

<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their aircraft type. I will therefore have a lookup table with two categories: Narrowbody and Widebody. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by Aircraft Type, by Tier. Then I have divided them by the total number of passengers by Aircraft Type. For example:
    - For NB aircrafts, and for Tier 1 passengers: In S19 we had 41,728 pax eligible for Tier 1 out of 7,222,830 pax flying on Narrowbody aircraft. This represents 0.6% of the costumers and I assume that this will be the number of costumers elegible for this specific Lounge in a future schedule.   

In [60]:
# your code here!

In [61]:
# columns that you want to group by
list_groupby = ['WB_NB_CAT','Lounge_eligibility_tier']

# grouping by WB_NB_CAT and Lounge_eligibility_tier
df_groupby_wb_nb = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

In [62]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_count
0,NB,Not eligible,9741188
1,NB,Tier 1,53292
2,NB,Tier 2,744829
3,NB,Tier 3,2240579
4,WB,Not eligible,5730922
5,WB,Tier 1,236775
6,WB,Tier 2,391264
7,WB,Tier 3,1523611


In [63]:
# getting the number of pax by aircraft type

# columns that you want to group by
list_groupby = ['WB_NB_CAT']

# grouping by WB_NB_CAT
df_groupby_wb_nb_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [64]:
# a look at the data
df_groupby_wb_nb_ttl

,WB_NB_CAT,pax_count_ttl
0,NB,12779888
1,WB,7882572


In [65]:
# now, let's left join df_groupby_wb_nb_ttl onto df_groupby_wb_nb
# this adds a new column to the df_groupby_wb_nb table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_wb_nb = pd.merge(df_groupby_wb_nb,
                            df_groupby_wb_nb_ttl,
                            on = ['WB_NB_CAT'],
                            how = 'left'
                           )

In [66]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_count,pax_count_ttl
0,NB,Not eligible,9741188,12779888
1,NB,Tier 1,53292,12779888
2,NB,Tier 2,744829,12779888
3,NB,Tier 3,2240579,12779888
4,WB,Not eligible,5730922,7882572
5,WB,Tier 1,236775,7882572
6,WB,Tier 2,391264,7882572
7,WB,Tier 3,1523611,7882572


In [67]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_wb_nb['pax_eligible%'] = (df_groupby_wb_nb['pax_count'] / df_groupby_wb_nb['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_wb_nb['pax_eligible%'] = df_groupby_wb_nb['pax_eligible%'].map('{:,.1f}%'.format)

In [68]:
# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_wb_nb.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [69]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_eligible%
0,NB,Not eligible,76.2%
1,NB,Tier 1,0.4%
2,NB,Tier 2,5.8%
3,NB,Tier 3,17.5%
4,WB,Not eligible,72.7%
5,WB,Tier 1,3.0%
6,WB,Tier 2,5.0%
7,WB,Tier 3,19.3%


In [70]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_wb_nb['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_wb_nb = df_groupby_wb_nb[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [71]:
# using set_index to come up with the final lookup table
df_groupby_wb_nb = df_groupby_wb_nb.set_index(['WB_NB_CAT','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [72]:
# final table
df_groupby_wb_nb

pax_eligible%              
Lounge_eligibility_tier        Tier 1 Tier 2 Tier 3
WB_NB_CAT                                          
NB                               0.4%   5.8%  17.5%
WB                               3.0%   5.0%  19.3%

<br>

**Feedback:** This analysis provides a lookup table in the format needed to be input into a future schedule. Nevertheless, this analysis is too high level and you haven't provided any evidence for your assumptions. To further enhance your answer use insights from the data and provide evidence for your assumptions. Please find some ideas below: 

- Using the same categories (WB,NB), plot data overtime to better understand the peaks for the different lounges. 
- Is there a way to split Widebody into more categories? Do the Haul, Region, Time of Day, or Country play a role in the number of passengers that are eligible in Tier 1, Tier 2 and Tier 3? Etc...

<br><br>

**Reasoning (Example 2)**:



<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their flight number. I will therefore have a lookup table with a lot of categories as each flight number is a category. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by flight number, by Tier. Then I have divided them by the total number of passengers flight number.  

In [73]:
# your code here!

In [74]:
# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_flt_no = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_count
0,5,HND,Not eligible,15825
1,5,HND,Tier 1,220
2,5,HND,Tier 2,888
3,5,HND,Tier 3,4469


In [75]:
# getting the number of pax by OPERATING_FLT_NO and DISCHARGE_STN_CD

# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD']

# grouping by WB_NB_CAT
df_groupby_flt_no_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [76]:
# a look at the data
df_groupby_flt_no_ttl.head(2)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,pax_count_ttl
0,5,HND,21402
1,5,NRT,38378


In [77]:
# now, let's left join df_groupby_flt_no_ttl onto df_groupby_flt_no
# this adds a new column to the df_groupby_flt_no table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_flt_no = pd.merge(df_groupby_flt_no,
                            df_groupby_flt_no_ttl,
                            on = ['OPERATING_FLT_NO','DISCHARGE_STN_CD'],
                            how = 'left'
                           )

In [78]:
# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_count,pax_count_ttl
0,5,HND,Not eligible,15825,21402
1,5,HND,Tier 1,220,21402
2,5,HND,Tier 2,888,21402
3,5,HND,Tier 3,4469,21402


In [79]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_flt_no['pax_eligible%'] = (df_groupby_flt_no['pax_count'] / df_groupby_flt_no['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_flt_no['pax_eligible%'] = df_groupby_flt_no['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_flt_no.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [80]:
# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_eligible%
0,5,HND,Not eligible,73.9%
1,5,HND,Tier 1,1.0%
2,5,HND,Tier 2,4.1%
3,5,HND,Tier 3,20.9%


In [81]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_flt_no['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_flt_no = df_groupby_flt_no[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [82]:
# using set_index to come up with the final lookup table
df_groupby_flt_no = df_groupby_flt_no.set_index(['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [83]:
# final table
df_groupby_flt_no.head(4)

pax_eligible%              
Lounge_eligibility_tier                  Tier 1 Tier 2 Tier 3
OPERATING_FLT_NO DISCHARGE_STN_CD                            
5                HND                       1.0%   4.1%  20.9%
                 NRT                       3.6%   4.8%  19.1%
7                HND                       3.3%   5.2%  19.8%
9                BKK                       0.3%   4.2%  15.9%

<br>

**Feedback:** This analysis goes at a very granular level, we might come up with missing values if we apply this lounge eligibility profiles to a future schedule. Here are some ideas to further enhance your answer: 

- What would happen if we fly to a new destination in the future? How do we ensure we have a lounge eligibility profile for this new route?
- As you can see in the example above: Pax eligible for Tier 1 for the BKK flight is significantly different from the rest. What's the most used aircraft type for this route? And why it differs that much from the rest? Is it because of the route characteristics instead?
- For SH routes, we might change the time of departure for a specific flight number from one year to the next. Explore the possibility of using a clasification that takes that into account, like using Time of Day instead of flight number. 

<br><br><br>

In [84]:
#--------------------------------
#
# Your turn!!!
#
#--------------------------------

<u>What level of granularity do you use?</u>
- ... (your answer here)

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- ... (your answer here)

In [85]:
# your code here



<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [86]:
# MORE TESTING PLEASE IGNORE AND DELETE BEFORE SUBMITTING

In [87]:
## more testing SCRAGG
df_final = df_lounge_elig_flight_info_country_acft_typ.copy()

In [88]:
df_final.shape

(1534750, 26)

In [89]:
df_lounge_eligibility.shape

(1534750, 12)

In [90]:
df_final.loc[df_final['DISCHARGE_STN_CD'] == 'BGI'].head(10)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
392,2023-08-20255,BA,255,2023-08-20,LHR,BGI,F,F,Silver,SAPP,Tier 1,1,5,777,M7,LHRBGI,10:30:00,NaN,NaN,NaN,NaN,WB,7.0,50.0,40.0,138.0
407,2023-04-20255,BA,255,2023-04-20,LHR,BGI,J,F,,,Tier 1,1,5,777,M7,LHRBGI,10:30:00,NaN,NaN,NaN,NaN,WB,7.0,50.0,40.0,138.0
968,2023-08-20255,BA,255,2023-08-20,LHR,BGI,M,M,,SAPP,Tier 3,3,5,777,M7,LHRBGI,10:30:00,NaN,NaN,NaN,NaN,WB,7.0,50.0,40.0,138.0
983,2023-04-20255,BA,255,2023-04-20,LHR,BGI,W,W,,EMER,Tier 2,1,5,777,M7,LHRBGI,10:30:00,NaN,NaN,NaN,NaN,WB,7.0,50.0,40.0,138.0
1544,2023-08-20255,BA,255,2023-08-20,LHR,BGI,F,F,Gold,EMER,Tier 1,5,5,777,M7,LHRBGI,10:30:00,NaN,NaN,NaN,NaN,WB,7.0,50.0,40.0,138.0
1559,2023-04-20255,BA,255,2023-04-20,LHR,BGI,W,W,Silver,,Tier 3,1,5,777,M7,LHRBGI,10:30:00,NaN,NaN,NaN,NaN,WB,7.0,50.0,40.0,138.0
2120,2023-08-20255,BA,255,2023-08-20,LHR,BGI,M,M,Gold,EMER,Tier 2,1,5,777,M7,LHRBGI,10:30:00,NaN,NaN,NaN,NaN,WB,7.0,50.0,40.0,138.0
2135,2023-04-20255,BA,255,2023-04-20,LHR,BGI,M,M,Silver,SAPP,Tier 3,4,5,777,M7,LHRBGI,10:30:00,NaN,NaN,NaN,NaN,WB,7.0,50.0,40.0,138.0
2696,2023-08-20255,BA,255,2023-08-20,LHR,BGI,M,M,,,Not eligible,129,5,777,M7,LHRBGI,10:30:00,NaN,NaN,NaN,NaN,WB,7.0,50.0,40.0,138.0
2711,2023-04-20255,BA,255,2023-04-20,LHR,BGI,J,J,Silver,,Tier 3,2,5,777,M7,LHRBGI,10:30:00,NaN,NaN,NaN,NaN,WB,7.0,50.0,40.0,138.0


In [91]:
df_final.loc[(df_final['BA_PAX_TIER']=='Silver')&(df_final['ONEWORLD_TIER']!='SAPP')].shape

(80888, 26)

In [92]:
df_final['STATUS'] = 'None'

In [93]:
# create new column for overall status calculation
df_final.loc[((df_final['BA_PAX_TIER'] == 'Silver')
              | (df_final['ONEWORLD_TIER'] == 'SAPP')
              , 'STATUS')] = 'Sapphire'


In [94]:
emerald_list = ['Gold For Life','Gold','Gold Guest List'
                ,'Gold Guest List For Life','Premier']


df_final.loc[((df_final['BA_PAX_TIER'].isin(emerald_list))
              | (df_final['ONEWORLD_TIER'] == 'EMER')
              , 'STATUS')] = 'Emerald'


In [95]:
df_final.head(4)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,STATUS
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,3,319,M4,LHROLB,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0,Emerald
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,3,319,M4,LHRSTR,16:35:00,DE,Germany,WEST EUROPE,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0,None
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,5,32A,H3,LHROSL,18:25:00,NO,Norway,SCANDINAVIA,EUROPE EXC UK,NB,0.0,20.0,0.0,150.0,None
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,5,319,A4,LHRNCE,14:40:00,FR,France,WEST EUROPE,EUROPE EXC UK,NB,0.0,16.0,0.0,119.0,Emerald


In [96]:
#df_final.to_csv('df_final_jamie.csv')

In [97]:
df_final.loc[df_final['Skew_Id'] == '2023-09-22502']

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,STATUS
243869,2023-09-22502,BA,502,2023-09-22,LHR,LIS,C,C,Gold,EMER,Tier 2,1,5,320,R3,LHRLIS,14:10:00,PT,Portugal,WEST EUROPE,EUROPE EXC UK,NB,0.0,28.0,0.0,139.0,Emerald
244445,2023-09-22502,BA,502,2023-09-22,LHR,LIS,C,C,Gold Guest List,EMER,Tier 1,1,5,320,R3,LHRLIS,14:10:00,PT,Portugal,WEST EUROPE,EUROPE EXC UK,NB,0.0,28.0,0.0,139.0,Emerald
245021,2023-09-22502,BA,502,2023-09-22,LHR,LIS,M,M,,EMER,Tier 2,1,5,320,R3,LHRLIS,14:10:00,PT,Portugal,WEST EUROPE,EUROPE EXC UK,NB,0.0,28.0,0.0,139.0,Emerald
245597,2023-09-22502,BA,502,2023-09-22,LHR,LIS,M,M,Silver,,Tier 3,2,5,320,R3,LHRLIS,14:10:00,PT,Portugal,WEST EUROPE,EUROPE EXC UK,NB,0.0,28.0,0.0,139.0,Sapphire
246173,2023-09-22502,BA,502,2023-09-22,LHR,LIS,C,C,,,Tier 3,24,5,320,R3,LHRLIS,14:10:00,PT,Portugal,WEST EUROPE,EUROPE EXC UK,NB,0.0,28.0,0.0,139.0,None
246749,2023-09-22502,BA,502,2023-09-22,LHR,LIS,M,M,,SAPP,Tier 3,3,5,320,R3,LHRLIS,14:10:00,PT,Portugal,WEST EUROPE,EUROPE EXC UK,NB,0.0,28.0,0.0,139.0,Sapphire
247325,2023-09-22502,BA,502,2023-09-22,LHR,LIS,M,C,Gold,EMER,Tier 2,1,5,320,R3,LHRLIS,14:10:00,PT,Portugal,WEST EUROPE,EUROPE EXC UK,NB,0.0,28.0,0.0,139.0,Emerald
247901,2023-09-22502,BA,502,2023-09-22,LHR,LIS,M,M,Gold,EMER,Tier 2,2,5,320,R3,LHRLIS,14:10:00,PT,Portugal,WEST EUROPE,EUROPE EXC UK,NB,0.0,28.0,0.0,139.0,Emerald
248477,2023-09-22502,BA,502,2023-09-22,LHR,LIS,M,C,Silver,SAPP,Tier 3,2,5,320,R3,LHRLIS,14:10:00,PT,Portugal,WEST EUROPE,EUROPE EXC UK,NB,0.0,28.0,0.0,139.0,Sapphire
249053,2023-09-22502,BA,502,2023-09-22,LHR,LIS,C,C,,EMER,Tier 2,2,5,320,R3,LHRLIS,14:10:00,PT,Portugal,WEST EUROPE,EUROPE EXC UK,NB,0.0,28.0,0.0,139.0,Emerald


In [98]:
# # writing algorithm to determine source of lounge elig.
# Confirmation that travel_cabin_cd controls lounge_tier.

# #cond1 = df_final['BOOKED_CABIN_CD'] == 'M'
# cond2 = df_final['TRAVEL_CABIN_CD'].isin(['J','C'])
# #cond3 = df_final['STATUS'] == 'None'
# cond4 = df_final['Lounge_eligibility_tier'] == 'Not eligible'
# conds =  cond2 & cond4
# not_elig_temp = df_final[conds]
# not_elig_temp.head(10)

# algorithm

# def calculate source (df):
    
#     #if 'status_source_ind' not in df.columns:
#     df['status_source_ind'] = 'NA'
#     df['cabin_source_ind'] = 'NA'
        
#     cond1 = 
    


SyntaxError: invalid syntax (<ipython-input-98-47d255713b52>, line 14)

In [99]:
# create new columns for year and time.
df_final.head()

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,STATUS
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,3,319,M4,LHROLB,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0,Emerald
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,3,319,M4,LHRSTR,16:35:00,DE,Germany,WEST EUROPE,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0,None
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,5,32A,H3,LHROSL,18:25:00,NO,Norway,SCANDINAVIA,EUROPE EXC UK,NB,0.0,20.0,0.0,150.0,None
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,5,319,A4,LHRNCE,14:40:00,FR,France,WEST EUROPE,EUROPE EXC UK,NB,0.0,16.0,0.0,119.0,Emerald
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,5,343,W9,LHRCAI,16:15:00,EG,Egypt,NORTH AFRICA,AFRICA,WB,0.0,45.0,0.0,212.0,None


In [100]:
# find the columns with nan values
nan_cols = df_final.isna()
cols_nans = nan_cols.any()
cols_nans = cols_nans[cols_nans]
print(cols_nans.index)

Index(['ACT_DEP_TML_CD', 'IATA_AC_TYP_CD', 'ACT_AC_TYP_CD', 'ROUTE_x',
       'GMT_PLND_DEP_TIME', 'COUNTRY_CD', 'COUNTRY_NM',
       'CORP_GEOG_CTRY_GRP_NM', 'CORP_GEOG_CONTINENT_NM', 'WB_NB_CAT',
       'FIRST_SEATS_QTY', 'CLUB_SEATS_QTY', 'PREM_ECONOMY_SEATS_QTY',
       'ECONOMY_SEATS_QTY'],
      dtype='object')


In [101]:
df_drop_nan_terminal_routes = df_final.dropna(subset=[
    'ACT_DEP_TML_CD','GMT_PLND_DEP_TIME'])
df_drop_nan_terminal_routes = df_drop_nan_terminal_routes.reset_index(drop=True)

In [102]:
df_drop_nan_terminal_routes['GMT_UPLIFT_DT'] = pd.to_datetime(df_drop_nan_terminal_routes['GMT_UPLIFT_DT'])
df_drop_nan_terminal_routes['Departure Year'] = pd.to_datetime(df_drop_nan_terminal_routes['GMT_UPLIFT_DT']).dt.year

df_drop_nan_terminal_routes['Departure Hour'] = pd.to_datetime(df_drop_nan_terminal_routes['GMT_PLND_DEP_TIME'],format='%H:%M:%S').dt.hour
df_drop_nan_terminal_routes.head(10)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,STATUS,Departure Year,Departure Hour
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,3,319,M4,LHROLB,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0,Emerald,2023,14
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,3,319,M4,LHRSTR,16:35:00,DE,Germany,WEST EUROPE,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0,None,2023,16
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,5,32A,H3,LHROSL,18:25:00,NO,Norway,SCANDINAVIA,EUROPE EXC UK,NB,0.0,20.0,0.0,150.0,None,2019,18
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,5,319,A4,LHRNCE,14:40:00,FR,France,WEST EUROPE,EUROPE EXC UK,NB,0.0,16.0,0.0,119.0,Emerald,2019,14
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,5,343,W9,LHRCAI,16:15:00,EG,Egypt,NORTH AFRICA,AFRICA,WB,0.0,45.0,0.0,212.0,None,2019,16
5,2023-08-05666,BA,666,2023-08-05,LHR,CHQ,C,C,Gold,EMER,Tier 2,3,5,32N,N3,LHRCHQ,09:45:00,GR,Greece,MEDITERRANEAN,EUROPE EXC UK,NB,0.0,20.0,0.0,150.0,Emerald,2023,9
6,2019-07-03227,BA,227,2019-07-03,LHR,ATL,J,J,Gold,EMER,Tier 2,4,5,777,V7,LHRATL,15:00:00,US,United States,USA,NORTH AMERICA,WB,14.0,48.0,40.0,124.0,Emerald,2019,15
7,2019-06-301448,BA,1448,2019-06-30,LHR,EDI,C,C,,SAPP,Tier 3,4,5,320,A3,LHREDI,14:20:00,GB,United Kingdom and Northern Ireland,UK,UK,NB,0.0,24.0,0.0,132.0,Sapphire,2019,14
8,2019-04-27886,BA,886,2019-04-27,LHR,OTP,M,M,,EMER,Tier 2,1,5,32N,N3,LHROTP,12:50:00,RO,Romania,EASTERN EUROPE,EUROPE EXC UK,NB,0.0,20.0,0.0,150.0,Emerald,2019,12
9,2023-10-01558,BA,558,2023-10-01,LHR,FCO,M,M,Gold,EMER,Tier 2,3,5,320,R3,LHRFCO,18:15:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,0.0,28.0,0.0,139.0,Emerald,2023,18


In [103]:
# group by year terminal, destination, lounge tier, status
list_groupby = ['ACT_DEP_TML_CD','DISCHARGE_STN_CD','Lounge_eligibility_tier','STATUS','Departure Hour','Departure Year']

df_groupby_tml_destination_status_year_hour = df_drop_nan_terminal_routes.groupby(list_groupby).agg(pax_count_ttl=('pax','sum')).reset_index()



df_groupby_tml_destination_status_year_hour.head(10)

df_groupby_tml_destination_status_year_hour.to_csv('tml_destination_status_year_hour.csv')

In [104]:
# filter by BA Premier
df_premier = df_final.loc[df_final['BA_PAX_TIER']=='Premier']

In [105]:
df_premier_stn.head(10)

NameError: name 'df_premier_stn' is not defined